In [108]:
with open('input.txt') as f:
    lines = f.readlines()

page_order_rules = []

for i, line in enumerate(lines):
    po = line.split('|')
    if len(po) == 2:
        page_order_rules.append((int(po[0]), int(po[1])))
    else:
        u_i = i
        break


print(page_order_rules[:5], page_order_rules[-5:])

updates = [[int(a) for a in l] for l in [x.strip().split(',') for x in lines[u_i+1:]]]

print(updates[0], updates[-1])

[(79, 26), (75, 21), (75, 86), (74, 69), (74, 86)] [(46, 85), (46, 33), (56, 58), (56, 53), (89, 85)]
[79, 72, 94, 83, 85] [79, 85, 36, 39, 95, 72, 64, 44, 73, 46, 54, 83, 65, 15, 38, 97, 94, 12, 47, 67, 35]


In [109]:
# create a lookup table for page order rules

por_lookup = {}

for x, i in page_order_rules:
    if x not in por_lookup:
        por_lookup[x] = [i]
    else:
        por_lookup[x].append(i)

print(por_lookup)
print(por_lookup[47])

{79: [26, 15, 52, 41, 57, 67, 18, 88, 33, 12, 37, 69, 56, 83, 44, 68, 89, 75, 74, 98, 48, 86, 34, 58], 75: [21, 86, 85, 89, 77, 52, 69, 48, 99, 26, 35, 82, 28, 46, 97, 36, 95, 39, 88, 58, 41, 53, 25, 72], 74: [69, 86, 75, 21, 18, 88, 99, 33, 89, 48, 53, 68, 34, 98, 57, 26, 25, 77, 41, 52, 56, 37, 28, 58], 35: [62, 74, 47, 57, 68, 12, 44, 38, 18, 94, 56, 79, 98, 67, 83, 15, 43, 33, 37, 54, 65, 73, 64, 34], 86: [28, 95, 53, 99, 46, 39, 43, 85, 69, 36, 77, 48, 97, 26, 82, 89, 38, 35, 58, 25, 47, 72, 62, 21], 28: [12, 25, 65, 83, 79, 43, 94, 35, 99, 36, 62, 39, 64, 95, 44, 82, 97, 46, 85, 47, 73, 38, 54, 72], 15: [52, 18, 33, 67, 88, 57, 26, 37, 74, 69, 86, 89, 21, 68, 53, 58, 75, 41, 77, 28, 98, 48, 34, 56], 95: [72, 97, 12, 15, 74, 85, 82, 73, 35, 94, 65, 39, 43, 64, 36, 79, 83, 47, 38, 54, 44, 46, 67, 62], 26: [38, 77, 47, 53, 97, 43, 21, 46, 95, 69, 36, 28, 35, 25, 72, 65, 39, 85, 54, 99, 64, 48, 62, 82], 73: [12, 52, 74, 86, 15, 57, 56, 68, 89, 88, 83, 58, 79, 37, 18, 98, 75, 26, 69, 

In [110]:
def check_update(update):
    for i in range(len(update)):
        # for all numbers after update[i], check if update[i] is in their not_after list
        after = update[i+1:]
        if len(after) > 0:
            # print(f"{update[i]}, looking at rules for {after}")
            disallowed = set([item for sublist in [por_lookup[x] for x in after if x in por_lookup] for item in sublist])
            # print(f"{update[i]} should not be in {disallowed}")
            if update[i] in disallowed:
                # print(f"Update {update[i]} is not allowed before {after}")
                unapproved_updates.append(update)
                return
    approved_updates.append(update)

approved_updates = []
unapproved_updates = []

for update in updates:
    check_update(update)

# find the middle number and sum them in all approved updates
print(sum([x[len(x)//2] for x in approved_updates]))

4462


In [111]:
# correctly order the unapproved_updates

def order_update(update):
    for i in range(len(update)):
        # for all numbers after update[i], check if update[i] is in their not_after list
        after = update[i+1:]
        if len(after) > 0:
            # print(f"{update[i]}, looking at rules for {after}")
            disallowed = set([item for sublist in [por_lookup[x] for x in after if x in por_lookup] for item in sublist])
            # print(f"{update[i]} should not be in {disallowed}")
            if update[i] in disallowed:
                # print(f"Update {update[i]} is not allowed before {after}")
                # swap with conflicting number
                # find index of number before update[i] in update that is in after
                swap_index = [j for j in range(len(update)) if update[j] in after][0]
                # print(f"Swapping {update[i]} with {update[swap_index]}")
                update[i], update[swap_index] = update[swap_index], update[i]

    return update
            
fixed_updates = []
while len(unapproved_updates) > 0:
    ordered_updates = [order_update(update) for update in unapproved_updates]
    unapproved_updates = []
    approved_updates = []

    for update in ordered_updates:
        check_update(update)
    
    fixed_updates.append(approved_updates)    

fixed_updates = [item for sublist in fixed_updates for item in sublist]
print(sum([x[len(x)//2] for x in fixed_updates]))

6767
